<a href="https://colab.research.google.com/github/jessica8403/TIL/blob/master/text%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [ ]:
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

import os, shutil

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import datasets

In [ ]:
!pip install konlpy

     |████████████████████████████████| 19.4 MB 1.5 MB/s 
     |████████████████████████████████| 448 kB 23.2 MB/s 


In [ ]:
%%shell
apt-get update
apt-get install g++ openjdk-8-jdk python-dev python3-dev
pip install JPype1
pip install konlpy

In [ ]:
%env JAVA_HOME "/usr/lib/jvm/java-8-openjdk-amd64"

env: JAVA_HOME="/usr/lib/jvm/java-8-openjdk-amd64"


In [ ]:
%%shell
bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)
pip install /tmp/mecab-python-0.996

In [ ]:
from konlpy.tag import Kkma, Komoran, Okt, Hannanum, Mecab

In [ ]:
# 교재에서는 실습을 위해서
samples = ['The cat sat on the mat.', 'The dog ate my homework.']

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
# 토크나이저 객체 생성(가장 빈도가 높은 단어 1,000개만 선택해서 토크나이징)
tokenizer = Tokenizer(num_words=1000)

# 단어 인덱스 구축
tokenizer.fit_on_texts(samples)

# 인덱스 리트스 확인
sequence = tokenizer.texts_to_sequences(samples)
print(sequence)

# 변환된 숫자 인덱스를 벡터 형태로 변환()
one_hot_encoded = tokenizer.texts_to_matrix(samples, mode='binary')
print(one_hot_encoded)

[[1, 2, 3, 4, 1, 5], [1, 6, 7, 8, 9]]
[[0. 1. 1. ... 0. 0. 0.]
 [0. 1. 0. ... 0. 0. 0.]]


In [ ]:
kkma = Kkma()
komoran = Komoran()
han = Hannanum()
mecab = Mecab()
okt = Okt()

In [ ]:
samples = ['아버지가방에들어가신다', '아기다리고기다리던데이터']

In [ ]:
kkma.pos(samples[0])

[('아버지', 'NNG'),
 ('가방', 'NNG'),
 ('에', 'JKM'),
 ('들어가', 'VV'),
 ('시', 'EPH'),
 ('ㄴ다', 'EFN')]

In [ ]:
kkma.morphs( samples[1] )

['아', '아', '기다리', '고', '기다리', '던데', '이', '터']

In [ ]:
kkma.nouns( samples[0] )

['아버지', '아버지가방', '가방']

In [ ]:
mecab.pos( samples[0] )

[('아버지', 'NNG'),
 ('가', 'JKS'),
 ('방', 'NNG'),
 ('에', 'JKB'),
 ('들어가', 'VV'),
 ('신다', 'EP+EC')]

In [ ]:
mecab.morphs( samples[1] )

['아기', '다리', '고', '기다리', '던', '데이터']

In [ ]:
mecab.nouns( samples[1] )

['아기', '데이터']

In [ ]:
komoran.pos( samples[1] )

[('아', 'IC'),
 ('기다리', 'VV'),
 ('고', 'EC'),
 ('기다리', 'VV'),
 ('던', 'ETM'),
 ('데이터', 'NNG')]

In [ ]:
samples_to_word = []
for sample in samples:
  # print( mecab.morphs(sample) )
  samples_to_word.append( mecab.morphs(sample) )

samples_to_word

[['아버지', '가', '방', '에', '들어가', '신다'], ['아기', '다리', '고', '기다리', '던', '데이터']]

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [ ]:
tokenizer = Tokenizer(num_words=1000)
tokenizer.fit_on_texts(samples_to_word)
sequence = tokenizer.texts_to_sequences(samples_to_word)
print(sequence) 

one_hot_encoded = tokenizer.texts_to_matrix(samples_to_word, mode='binary')
print(one_hot_encoded)

[[1, 2, 3, 4, 5, 6], [7, 8, 9, 10, 11, 12]]
[[0. 1. 1. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [ ]:
from tensorflow.keras.datasets import imdb

In [ ]:
# 특성으로 사용할 단어의 갯수
max_features = 10000
# 사용할 텍스트의 길이
maxLen = 20

# 정수 리스트로 데이터를 로드
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

# 정수 리스트를 (samples, maxLen) 크기의 2D 정수 텐서로 변환
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words = max_features)

In [ ]:
import keras

In [ ]:
x_train = keras.preprocessing.sequence.pad_sequences(x_train, maxlen=maxLen)
x_test = keras.preprocessing.sequence.pad_sequences(x_test, maxlen=maxLen)

In [ ]:
x_train

array([[  65,   16,   38, ...,   19,  178,   32],
       [  23,    4, 1690, ...,   16,  145,   95],
       [1352,   13,  191, ...,    7,  129,  113],
       ...,
       [  11, 1818, 7561, ...,    4, 3586,    2],
       [  92,  401,  728, ...,   12,    9,   23],
       [ 764,   40,    4, ...,  204,  131,    9]], dtype=int32)

In [ ]:
model = tf.keras.models.Sequential()

# 10000개의 단어에 대해서 8차원의 임베딩을 학습
model.add( tf.keras.layers.Embedding(10000, 8, input_length=maxLen ) )

# 3d 임베딩 벡터를 (samples, maxLen * 8 ) 크기의 2D 벡터로 변환
model.add( tf.keras.layers.Flatten() )

# 분류기 추가
model.add( tf.keras.layers.Dense(1, activation='sigmoid'))

# 최적화 옵션
model.compile(
  loss = 'binary_crossentropy',
  metrics = ['acc'],
  optimizer = 'rmsprop'
)

In [ ]:
hist = model.fit( x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
625/625 [==============================] - 7s 7ms/step - loss: 0.6743 - acc: 0.6041 - val_loss: 0.6298 - val_acc: 0.6916
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.5510 - acc: 0.7477 - val_loss: 0.5328 - val_acc: 0.7292
Epoch 3/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4655 - acc: 0.7863 - val_loss: 0.5038 - val_acc: 0.7438
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4232 - acc: 0.8076 - val_loss: 0.4958 - val_acc: 0.7496
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3951 - acc: 0.8228 - val_loss: 0.4962 - val_acc: 0.7528
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3724 - acc: 0.8354 - val_loss: 0.4993 - val_acc: 0.7534
Epoch 7/10
625/625 [==============================] - 3s 5ms/step - loss: 0.3528 - acc: 0.8468 - val_loss: 0.5039 - val_acc: 0.7566
Epoch 8/10
625/625 [==============================] - 3s 5ms/step - loss: 0.

In [ ]:
# dir_path = '/content/drive/MyDrive/멀티캠퍼스/data/IMDB'
# file_path = '/content/drive/MyDrive/멀티캠퍼스/data/IMDB/aclImdb.zip'

# # 압축이 풀리면 다음에 다시 압축을 풀지 않도록 주석으로 바꿔주도록 합니다.
# %cd '{dir_path}'
# !unzip '{file_path}'

In [42]:
dir_path = '/content/drive/MyDrive/멀티캠퍼스/data/IMDB/aclImdb'

file_path = os.path.join( dir_path, 'train/neg')
file_name = os.listdir(file_path)[0]
with open( os.path.join(file_path, file_name), 'r') as f:
  print(f.read())

Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.


In [44]:
dir_path = '/content/drive/MyDrive/멀티캠퍼스/data/IMDB/aclImdb'
train_dir_path = os.path.join( dir_path, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
  dir_name = os.path.join( train_dir_path, label_type )
  for fname in os.listdir( dir_name ):
    # 폴더 내에 .txt가 아닌 파일도 있는 것 같습니다.
    if fname[-4:] == '.txt':
      with open( os.path.join( dir_name, fname), 'r' ) as f:
        texts.append( f.read() )
      if label_type == 'neg': labels.append(0)
      else: labels.append(1)

In [45]:
texts[:2]

["Story of a man who has unnatural feelings for a pig. Starts out with a opening scene that is a terrific example of absurd comedy. A formal orchestra audience is turned into an insane, violent mob by the crazy chantings of it's singers. Unfortunately it stays absurd the WHOLE time with no general narrative eventually making it just too off putting. Even those from the era should be turned off. The cryptic dialogue would make Shakespeare seem easy to a third grader. On a technical level it's better than you might think with some good cinematography by future great Vilmos Zsigmond. Future stars Sally Kirkland and Frederic Forrest can be seen briefly.",
 "Airport '77 starts as a brand new luxury 747 plane is loaded up with valuable paintings & such belonging to rich businessman Philip Stevens (James Stewart) who is flying them & a bunch of VIP's to his estate in preparation of it being opened to the public as a museum, also on board is Stevens daughter Julie (Kathleen Quinlan) & her son.

In [46]:
len(texts)

25000

In [48]:
labels[:2]

[0, 0]

In [49]:
from tensorflow.keras.preprocessing.text import Tokenizer

In [50]:
maxLen = 100 # 100개의 단어만 사용
max_words = 10000 # 주어진 학습 셋에서 빈도가 가장 높은 10000개의 단어만 사용

training_samples = 200
validation_samples = 10000 # 일부러 과적합 하려고 한 것 같습니다.

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequence = tokenizer.texts_to_sequences(texts)

# 정수 리스트를 (samples, maxLen) 크기의 2D 정수 텐서로 변환
data = keras.preprocessing.sequence.pad_sequences( sequence, maxlen = maxLen )
labels = np.asarray( labels )

In [51]:
data.shape 

(25000, 100)

In [52]:
labels.shape

(25000,)

In [54]:
# 부정 리뷰 이후에 긍정 리뷰가 이어지기 때문에
# 샘플링 하기 위해서 데이터를 무작위로 섞은 이후에 200개의 샘플을 추출

# 부정 리뷰 이후에 긍정 리뷰가 이어지기 때문에
# 샘플링 하기 위해서 데이터를 무작위로 섞은 이후에 200개의 샘플을 추출

# 전체 데이터의 개수만큼 수열을 생성
indices = np.arange( data.shape[0] ) 
print( indices )

# 생성된 수열을 섞어줍니다.
np.random.shuffle( indices )
print( indices )

data = data[indices]
labels = labels[indices]

# 학습자료와 검증 자료로 분할
x_train = data[:training_samples]
y_train = labels[:training_samples]
x_validation = data[training_samples: training_samples + validation_samples]
y_validation = labels[training_samples: training_samples + validation_samples]

[    0     1     2 ... 24997 24998 24999]
[ 2073 17619 22198 ...  3774  5499  1155]


In [55]:
print(x_train.shape)
print(x_validation.shape)

(200, 100)
(10000, 100)


In [56]:
y_train

array([0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 0, 1, 0,
       0, 0, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0,
       1, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1,
       1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1,
       1, 1, 1, 1, 0, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1,
       0, 1])